<a href="https://colab.research.google.com/github/sayantanig/noisy_labels/blob/master/Fractional_Imputation_Testing_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
from numpy.random import default_rng

import pandas as pd

import torch
import torch.nn as nn

from random import sample
from random import random

from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis as QDA
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn import metrics

import matplotlib.pyplot as plt

import math
import matplotlib.pyplot as plt

In [ ]:
# Global variables

trainX = 0      # training data - values
trainY = 0      # training data - labels
testX = 0       # test data - values
testY = 0       # test data - labels
datasetX = []
datasetY = []
n = 500      # number of samples
sample_ratio = 0.2  # test:train ratio
train_size = int((1-sample_ratio)*n)  # size of training data after split
num_labels = 2  # number of labels
y_dim = 2       # dimensions of Y
delta = 0       # missing Y indicators
boot_num = 10   # number of bootstraps
x_boot_dim = 3  # dimensions of bootstraps for X
x_dim = 6       # dimensions of X
num_posbl_x = 64
x_proba=[[0 for i in range(num_posbl_x)] for j in range(boot_num)]
num_posbl_y = pow(num_labels, y_dim)
pi_est_arr = [[[1/num_posbl_y for k in range(num_posbl_y)] for i in range(num_posbl_x)] for j in range(boot_num)]
pi_plus_arr = [[[[1/num_posbl_y for l in range(num_posbl_y)] for k in range(pow(2,y_dim))] for i in range(num_posbl_x)] for j in range(boot_num)]
w = [0 for i in range(boot_num)]  # model weights

In [ ]:
# Generating X dataset

def generateX():
  X = [0 for j in range(x_dim)]
  for i in range(x_dim):
    X[i] = np.random.choice([1,2,3,4], n, p=[0.25,0.25,0.25,0.25])

  X = np.transpose(np.asarray(X))
  # print(X)

  return X

In [ ]:
datasetX = generateX()
# print(*datasetX, sep='\n')

In [ ]:
# Generating Y dataset

def generateY():
  Y = []
  for i in datasetX:
    temp = i[0]+i[1]+i[2]
    # # print (i[0], i[1], i[2], temp)
    # if temp<=5:
    #   y1=np.random.choice([0,1], 1, p=[0.9,0.1])[0]
    #   # y2_p=(0.8+y1)/2
    #   y2=np.random.choice([0,1], 1, p=[0.9,0.1])[0]
    # elif temp>5 and temp<=7:
    #   y1=np.random.choice([0,1], 1, p=[0.6,0.4])[0]
    #   # y2_p=0.8*y1+0.1
    #   y2=np.random.choice([0,1], 1, p=[0.6,0.4])[0]
    # elif temp>7 and temp<=9:
    #   y1=np.random.choice([0,1], 1, p=[0.3,0.7])[0]
    #   # y2_p=0.9-0.5*y1
    #   y2=np.random.choice([0,1], 1, p=[0.3,0.7])[0]
    # else:
    #   y1=np.random.choice([0,1], 1, p=[0.1,0.9])[0]
    #   # y2_p=0.6
    #   y2=np.random.choice([0,1], 1, p=[0.1,0.9])[0]
    # y2=np.random.choice([0,1], 1, p=[1-y2_p,y2_p])[0]
    if temp<8:
      y1=1
      y2=1
    else:
      y1=0
      y2=0
    Y.append([y1,y2])
  # Y

  return Y

In [ ]:
datasetY = generateY()
# print(*datasetY, sep='\n')

In [ ]:
# Train and test setup

def data_split():
  trainX, testX, trainY, testY = train_test_split(datasetX, datasetY, test_size=sample_ratio, random_state=0)
  train_size = len(trainX)

  return (trainX, trainY, testX, testY)

In [ ]:
trainX, trainY, testX, testY = data_split()

In [ ]:
# for i in range(len(trainX)):
#   print(trainX[i], trainY[i])

In [ ]:
# Generate delta

def delta_func():
  delta=[[1 for j in range(y_dim)] for i in range(n)]
  
  for i in range(n):
    for j in range(y_dim):
      delta[i][0]=np.random.choice([0,1], 1, p=[0.2,0.8])[0]
      delta[i][1]=np.random.choice([0,1], 1, p=[0.4,0.6])[0]
  
  return delta

In [ ]:
delta = delta_func()
# print(*delta, sep='\n')

In [ ]:
# Bootstrapping

def bootstrap():
  b = [0 for j in range(boot_num)]
  for i in range(boot_num):
    b_x = np.random.choice(train_size, size=train_size, replace = True)
    b_x.sort()
    b[i] = b_x[:]

  b_indices = [0 for j in range(boot_num)]
  # for i in range(boot_num):
    # b_indices_x = np.random.choice(x_dim, size=x_boot_dim, replace = False)
    # b_indices_x.sort()
    # b_indices_x[0] = 0
    # b_indices[i] = b_indices_x[:]

  b_indices[0] = [0, 1, 2]
  b_indices[1] = [0, 1, 3]
  b_indices[2] = [0, 1, 4]
  b_indices[3] = [0, 1, 5]
  b_indices[4] = [0, 2, 3]
  b_indices[5] = [0, 2, 4]
  b_indices[6] = [0, 2, 5]
  b_indices[7] = [0, 3, 4]
  b_indices[8] = [0, 3, 5]
  b_indices[9] = [3, 4, 5]

  oob = [[] for j in range(boot_num)]
  for i in range(boot_num):
    index = 0
    for num in range(len(trainX)):
      if num > b[i][-1] or num < b[i][index]:
        oob[i].append(num)
      while index<len(b[i]) and b[i][index] == num:
        index+=1

  return (b, b_indices, oob)

In [ ]:
(b, b_indices, oob) = bootstrap()
# print("Boot:")
# print(*b, sep='\n')
print("\nBoot indices: ")
print(*b_indices, sep='\n')
# print("\nOut of bag: ")
# print(*oob, sep='\n')


Boot indices: 
[0, 1, 2]
[0, 1, 3]
[0, 1, 4]
[0, 1, 5]
[0, 2, 3]
[0, 2, 4]
[0, 2, 5]
[0, 3, 4]
[0, 3, 5]
[3, 4, 5]


In [ ]:
def compute_hash_x(arr_x):
  # return (arr_x[0]-1)
  # return (arr_x[0]-1)*4 + (arr_x[1]-1)
  return (arr_x[0]-1)*16 + (arr_x[1]-1)*4 + (arr_x[2]-1)

In [ ]:
def compute_x_proba():
  global x_proba
  x_proba=[[0 for i in range(num_posbl_x)] for j in range(boot_num)]
  for i in range(boot_num):
    for j in range(len(trainX)):
      first_dgt = trainX[b[i][j]][b_indices[i][0]]
      # x_ind = compute_hash_x([first_dgt])
      second_dgt = trainX[b[i][j]][b_indices[i][1]]
      # x_ind = compute_hash_x([first_dgt, second_dgt])
      third_dgt = trainX[b[i][j]][b_indices[i][2]]
      x_ind = compute_hash_x([first_dgt, second_dgt, third_dgt])

      x_proba[i][x_ind]+=1
    
    for j in range(num_posbl_x):
      x_proba[i][j]/=len(trainX)

In [ ]:
# Full sample estimator

def compute_fs_estimator():
  theta_fs = [0 for i in range(y_dim)]

  fullX = np.concatenate((np.array(trainX), np.array(testX)))
  fullY = np.concatenate((np.array(trainY), np.array(testY)))
  
  for j in range(y_dim):
    for i in fullY:
      theta_fs[j] += i[j]
    theta_fs[j]/= n

  # print("Full sample estimator: ",theta_fs)
  
  return theta_fs

In [ ]:
# fs_estimator = compute_fs_estimator()
# print(fs_estimator)

In [ ]:
# Complete case estimator

def compute_cc_estimator():
  theta_cc = [0 for i in range(y_dim)]
  n_y = [0 for i in range(y_dim)] 

  fullX = np.concatenate((np.array(trainX), np.array(testX)))
  fullY = np.concatenate((np.array(trainY), np.array(testY)))

  for j in range(y_dim):
    for i in range(len(fullX)):
      theta_cc[j] += fullY[i][j] * delta[i][j]
      n_y[j] += delta[i][j]
    theta_cc[j] /= n_y[j]

  # print("Complete case estimator: ",theta_cc)

  return theta_cc

In [ ]:
# cc_estimator = compute_cc_estimator()
# print(cc_estimator)

* Missing pattern index (a): binary string converted to integer
* Label index (b): index of label in alphabetic order
* Retuns: index of the label where the letters of b corresponding to the 0s of a are replaced by the alphabetically lowest label.  
* eg: given a: index(00111), b: index(abacd), returns index(aaacd)


In [ ]:
# Hashing function

def reduce_mult(a,b):
  # a:missing pattern index, b:label index
  num=0
  p=0
  while(a!=0):
    dgt_a=a%2
    a=a//2
    dgt_b=b%num_labels
    b=b//num_labels
    num += dgt_a*dgt_b*pow(num_labels, p)
    p+=1
  return int(num) # tODO: figure out why this might return float

In [ ]:
# 

def concat_arr(a):
  # a:missing pattern as a binary array
  mp = ""
  for i in a:
    mp += str(i)
  return mp

In [ ]:
# 

def y_index(y_arr):
  num=0
  pow=0
  for i in range(len(y_arr)-1,-1,-1):
    num+=y_arr[i]*math.pow(num_labels,pow)
    pow+=1
  return num

In [ ]:
# Updating E step for EM algorithm

def n_update(pi_est, pi_plus_est, n_plus):
  n_new = [[0 for i in range(num_posbl_y)] for j in range(pow(2, y_dim))]

  for i in range(pow(2,y_dim)):
    for j in range(num_posbl_y):
      try:
        n_new[i][j]=n_plus[i][reduce_mult(i,j)]*(pi_est[j]/pi_plus_est[i][reduce_mult(i,j)])
      except:
        if(n_plus[i][reduce_mult(i,j)] == 0 and pi_est[j]==0 and pi_plus_est[i][reduce_mult(i,j)]==0):
          n_new[i][j]=0
        else:
          print("i", i, "j", j, "reduce_mult(i,j)", reduce_mult(i,j))
          print("n_plus", n_plus)
          print("pi_est", pi_est)
          print("pi_plus_est", pi_plus_est)
          assert(1==0)
  return n_new

In [ ]:
# Updating M step for EM algorithm

def pi_update(pi_est, n_est, n_curr):
  error_max=0
  pi_new = [0 for i in range(num_posbl_y) ]

  for i in range(pow(2, y_dim)):
    for j in range(num_posbl_y):
      pi_new[j]+=n_est[i][j]

  for j in range(num_posbl_y):
    # print(n_curr)
    pi_new[j] /= n_curr
    error_max=max(error_max, abs(pi_est[j]-pi_new[j]))

  return (pi_new, error_max)

In [ ]:
# Updating Pi+ (M step) for EM algorithm

def pi_plus_update(pi_est):
  pi_plus = [[0 for i in range(num_posbl_y)]  for j in range(pow(2,y_dim))]
  
  # print(pi_est)
  for i in range(pow(2,y_dim)):
    for j in range(num_posbl_y):
      pi_plus[i][reduce_mult(i,j)] += pi_est[j]
  
  return pi_plus

In [ ]:
# EM algorithm compute E and M step until convergence

def em_alg(trainY_curr, index_curr):
  # print("Method: EM Algorithm")
  
  n_curr = len(trainY_curr)

  n_plus = [[0 for i in range(num_posbl_y)] for j in range(pow(2, y_dim))]
  for i in range(n_curr):
    st_row = ""
    st_col = ""
    for j in range(y_dim):
      ind = index_curr[i]
      # print(ind)
      st_row += str(delta[ind][j])
      st_col += str(trainY_curr[i][j]*delta[ind][j])
    n_plus[int(st_row, 2)][int(st_col, num_labels)] += 1

  pi_est = [1/num_posbl_y for i in range(num_posbl_y)]
  error_curr = 1
  stop_eps = 0.00001

  while(error_curr > stop_eps):
    pi_plus_est = pi_plus_update(pi_est)
    n_est =  n_update(pi_est, pi_plus_est, n_plus)
    (pi_est, error_curr) = pi_update(pi_est, n_est, n_curr)
    
    # print("Pi_est: "+ pi_est)

  return pi_est

In [ ]:
# Compute log likelihood

def log_likelihood(i):  # i: boot index
  # print("Method: log_likelihood")

  # global trainY_arr
  trainY_arr = [[] for j in range(num_posbl_x)]
  index_arr = [[] for j in range(num_posbl_x)]
  for j in range(train_size):
    first_dgt = trainX[b[i][j]][b_indices[i][0]]
    # x_ind = compute_hash_x([first_dgt])
    second_dgt = trainX[b[i][j]][b_indices[i][1]]
    # x_ind = compute_hash_x([first_dgt, second_dgt])
    third_dgt = trainX[b[i][j]][b_indices[i][2]]
    x_ind = compute_hash_x([first_dgt, second_dgt, third_dgt]) # generalize this
    trainY_arr[x_ind].append(trainY[b[i][j]])
    index_arr[x_ind].append(j)
    # print(trainY_arr)

  compute_x_proba()
  for j in range(num_posbl_x):
    if(len(trainY_arr[j])!=0):
      pi_est_arr[i][j] = em_alg(trainY_arr[j], index_arr[j])
    for k in range(len(pi_est_arr[i][j])):
      pi_est_arr[i][j][k]*=x_proba[i][j]
    pi_plus_arr[i][j] = pi_plus_update(pi_est_arr[i][j])

  ll = 0
  for j in range(len(oob[i])):
    first_dgt = trainX[oob[i][j]][b_indices[i][0]]
    # x_ind = compute_hash_x([first_dgt])
    second_dgt = trainX[oob[i][j]][b_indices[i][1]]
    # x_ind = compute_hash_x([first_dgt, second_dgt])
    third_dgt = trainX[oob[i][j]][b_indices[i][2]]
    x_ind = compute_hash_x([first_dgt, second_dgt, third_dgt])  # generalize this
    mp_index = int("1"*y_dim, 2) # dummy missing pattern index
    y_ind = y_index([trainY[oob[i][j]][k]*delta[oob[i][j]][k] for k in range(y_dim)])
    # print(type(x_ind), type(mp_index), type(reduce_mult(mp_index,y_ind)))
    try:
      ll += math.log(pi_plus_arr[i][x_ind][mp_index][reduce_mult(mp_index, y_ind)])
    except:
      pass
      # print (reduce_mult(mp_index, y_ind))
      # print (pi_plus_arr[i][x_ind][mp_index][reduce_mult(mp_index, y_ind)])
      # print (math.log(pi_plus_arr[i][x_ind][mp_index][reduce_mult(mp_index, y_ind)]))
  ll/=len(oob[i])

  return ll

In [ ]:
# Compute model weights from log likelihood

def model_weights(lamda, ll):
  # print("Method: model_weights")

  denom = 0
  # w = [0 for j in range(boot_num)]
  for j in range(boot_num):
    w[j] = math.exp(ll[j]*lamda)
    denom += w[j]
    # print(j, " -> ",w[j]," -> ",denom)
 
  for j in range(boot_num):
    w[j] = w[j]/denom
  return w

In [ ]:
def compute_conditional_boot_pi(currX, currY):
  # print("Method: compute_conditional_boot_pi")

  conditional_boot_pi = np.zeros((boot_num, len(currX), num_posbl_y))
  for j in range(boot_num):
    for i in range(len(currX)):
      X = currX[i]  
      Y = currY[i]
      y_ind_actual = int(concat_arr(Y), num_labels)
      mp_ind = int(concat_arr(delta[i]), 2)
      first_dgt = X[b_indices[j][0]]
      # x_ind = compute_hash_x([first_dgt])
      second_dgt = X[b_indices[j][1]]
      # x_ind = compute_hash_x([first_dgt, second_dgt])
      third_dgt = X[b_indices[j][2]]
      x_ind = compute_hash_x([first_dgt, second_dgt, third_dgt])    # generalize this
      for y_ind in range(num_posbl_y):
        if reduce_mult(mp_ind, y_ind) != reduce_mult(mp_ind, y_ind_actual):
          conditional_boot_pi[j][i][y_ind] = 0
          continue
        try:
          conditional_boot_pi[j][i][y_ind] = pi_est_arr[j][x_ind][y_ind]/pi_plus_arr[j][x_ind][mp_ind][reduce_mult(mp_ind,y_ind)]
        except:
          pass    # to avoid division by 0 error
          # print(conditional_boot_pi[j][i][y_ind])
          # print("pi_est_arr", pi_est_arr)
          # print(pi_est_arr[j][x_ind][y_ind])
          # print(pi_plus_arr[j][x_ind][mp_ind][reduce_mult(mp_ind,y_ind)])
          
  # print(conditional_boot_pi)
  return conditional_boot_pi

In [ ]:
def compute_conditional_pi(conditional_boot_pi, currX, currY):
  # print("Method: compute_conditional_pi")

  conditional_pi = np.zeros((len(currX), num_posbl_y))
  for i in range(len(currX)):
    # X = currX[i]
    # Y = currY[i]
    # first_dgt = X[b_indices[j][0]]
    # second_dgt = X[b_indices[j][1]]
    # third_dgt = X[b_indices[j][2]]
    # x_ind = compute_hash_x([first_dgt, second_dgt, third_dgt])
    for y_ind in range(num_posbl_y):
      for j in range(boot_num):
        conditional_pi[i][y_ind] += w[j]*conditional_boot_pi[j][i][y_ind]
  # print("conditional_pi",conditional_pi)
  return conditional_pi

In [ ]:
# pi{hat}_a in the doc

def compute_wt_pi_est(conditional_pi, currX, currY):
  # print("Method: compute_wt_pi_est")

  wt_pi_est = np.zeros(num_posbl_y)
  for i in range(num_posbl_y): 
    for j in range(len(currX)):
      wt_pi_est[i]+=conditional_pi[j][i]/len(currX)
  # print("wt_pi_est",wt_pi_est)
  return wt_pi_est

In [ ]:
# pi{tilda}_k in the doc

def compute_partial_wt_pi_est(wt_pi_est, currX, currY):
  # print("Method: compute_partial_wt_pi_est")

  partial_wt_pi_est = np.zeros((pow(2,y_dim), num_posbl_y))
  for i in range(pow(2,y_dim)):
    for j in range(num_posbl_y):
      partial_wt_pi_est[i][reduce_mult(i,j)] += wt_pi_est[j]
  # print(partial_wt_pi_est)
  return partial_wt_pi_est

In [ ]:
# Validation_log_likelihood

def validation_log_likelihood(partial_wt_pi_est, lamda):
  # print("Method: validation_log_likelihood")

  v_ll = 0 
  for i in range(len(testX)):
    delta_ind = int(concat_arr(delta[len(trainX) + i]), 2)
    y_ind = int(concat_arr(testY[i]), 2)
    # print(math.log(partial_wt_pi_est[delta_ind][reduce_mult(delta_ind,y_ind)]))
    try:
      v_ll += math.log(partial_wt_pi_est[delta_ind][reduce_mult(delta_ind, y_ind)])
    except:
      pass
  # print("lambda", lamda, "v_ll",v_ll)
  return v_ll

In [ ]:
def compute_tuning_parameter():
  # print("Method: compute_tuning_parameter")

  lam = np.linspace(0.0, 10.0, 100)  # generate 100 lambda values from 0.0 to 10
  ll = np.zeros(boot_num) #[0]*boot_num
  v = -math.inf
  optimal_lamda = 0

  for i in range(boot_num):
    # print(i)
    ll[i] = log_likelihood(i)

  print("Log likelihood: ")
  print(ll)

  for lamda in lam:
    # print("Current lambda: ", lamda)
    w = model_weights(lamda, ll)
    # print("weights_lambda",w)
    conditional_boot_pi = compute_conditional_boot_pi(trainX, trainY)
    # print("Conditional Boot Pi: ", conditional_boot_pi)
    conditional_pi = compute_conditional_pi(conditional_boot_pi, trainX, trainY)
    # print("Conditional Pi: ", conditional_pi)
    wt_pi_est = compute_wt_pi_est(conditional_pi, trainX, trainY)
    # print("Wt Pi Est sum: ", sum(wt_pi_est))
    partial_wt_pi_est = compute_partial_wt_pi_est(wt_pi_est, trainX, trainY)
    # print("Partial Wt Pi Est: ", partial_wt_pi_est)
    v_ll = validation_log_likelihood(partial_wt_pi_est, lamda)
    # print("Validation Log Likelihood: ", v_ll)
    # print("Current Lambda: ", lamda)
    if(v_ll > v):
      v = v_ll
      optimal_lamda = lamda

  w = model_weights(optimal_lamda, ll)


  # print("Validation log likelihood: ", v_ll)

  print("Model weights: ")
  print(*w, sep='\n')

  for i in range(len(w)):
    w_avg[i] += w[i]

  print("Optimal tuning parameter: ", optimal_lamda)

  return optimal_lamda

In [ ]:
# Find optimal weight from lamda

def optimal_weight():
  # print("Method: optimal_weight")
  
  optimal_lamda = compute_tuning_parameter()
  fullX = np.concatenate((np.array(trainX), np.array(testX)))
  fullY = np.concatenate((np.array(trainY), np.array(testY)))
  conditional_boot_pi = compute_conditional_boot_pi(fullX, fullY)
  conditional_pi = compute_conditional_pi(conditional_boot_pi, fullX, fullY)

  # print("Optimal weight: ", conditional_pi)

  return conditional_pi

In [ ]:
def compute_fi_estimator():
  # print("Method: compute_fi_estimator")
  
  theta_fi = [0 for i in range(y_dim)]
  conditional_pi = optimal_weight()

  for y_estimates in conditional_pi:
    theta_fi[0] += (y_estimates[2] + y_estimates[3])
    theta_fi[1] += (y_estimates[1] + y_estimates[3])
  try:
    theta_fi[0] /= n
    theta_fi[1] /= n
  except:
    pass

  return theta_fi

In [ ]:
theta_fs, theta_cc, theta_fi = [],[],[]   # Estimators

iter = 100    # number of iterations
w_avg = [0 for i in range(boot_num)] # average model weights over the number of iterations

for i in range(iter):
  print("\nLoop: ", i)
  print("------------")
  datasetX = generateX()
  datasetY = generateY()

  delta = delta_func()
  # if min(np.sum(np.array(delta), axis=0))==0:
  #   continue
  (trainX, trainY, testX, testY) = data_split()
  pi_est_arr = [[[1/num_posbl_y for k in range(num_posbl_y)] for i in range(num_posbl_x)] for j in range(boot_num)]
  pi_plus_arr = [[[[1/num_posbl_y for l in range(num_posbl_y)] for k in range(pow(2,y_dim))] for i in range(num_posbl_x)] for j in range(boot_num)]

  fs = compute_fs_estimator()
  cc = compute_cc_estimator()
  fi = compute_fi_estimator()

  # print(fs, cc, fi)

  theta_fs.append(fs)
  theta_cc.append(cc)
  theta_fi.append(fi)
  
  fs_mean = np.mean(np.array(theta_fs), axis=0)
  cc_mean = np.mean(np.array(theta_cc), axis=0)
  fi_mean = np.mean(np.array(theta_fi), axis=0)

  print("Current mean: ")
  print("Mean 1: ", fs_mean[0], cc_mean[0], fi_mean[0])
  print("Mean 2: ", fs_mean[1], cc_mean[1], fi_mean[1])

  fs_var = np.var(np.array(theta_fs), axis=0)
  cc_var = np.var(np.array(theta_cc), axis=0)
  fi_var = np.var(np.array(theta_fi), axis=0)

  print("Current variance: ")
  print("Var 1: ", fs_var[0], cc_var[0], fi_var[0])
  print("Var 2: ", fs_var[1], cc_var[1], fi_var[1])


Loop:  0
------------
Log likelihood: 
[ -6.47185518  -7.27329972  -7.13927747 -16.04915284  -7.76140531
  -7.20755318  -6.71700269  -8.29715301  -7.54003829 -10.73200043]
Model weights: 
0.9187598562549225
0.00030378939118945805
0.0011604430458475809
2.3416758913516037e-42
2.3054566845972876e-06
0.0005862815233065135
0.07916622029988844
1.0865113844275315e-08
2.1093163047021955e-05
2.89485139855198e-19
Optimal tuning parameter:  10.0

Loop:  1
------------
Log likelihood: 
[ -7.54819615  -8.17706492  -9.70608373 -10.72685589  -7.36636487
  -7.97644023  -6.67623501  -8.69912984  -8.15574244  -6.70202494]
Model weights: 
0.1
0.1
0.1
0.1
0.1
0.1
0.1
0.1
0.1
0.1
Optimal tuning parameter:  0.0

Loop:  2
------------
Log likelihood: 
[-6.44085906 -7.21882877 -7.43067016 -6.88903937 -6.42626159 -6.19569083
 -8.67117711 -9.04708652 -7.64324896 -7.43640348]
Model weights: 
0.2184498242842424
0.033134305327205515
0.01982639820000192
0.07370411402223569
0.22631869572321206
0.3957978336291435
0.

In [ ]:
for i in range(len(w_avg)):
  w_avg[i] = w_avg[i]/iter

print("Average model weights: ")
print(*w_avg, sep='\n')

print("Sum: ", sum(w_avg))

Average model weights: 
0.320098218857488
0.061095758658056214
0.07496975683930311
0.09441366106534646
0.1358946890472817
0.08681462474695519
0.08724055872650661
0.04269149024281096
0.059651309232229346
0.037129932584022154
Sum:  0.9999999999999998


In [ ]:
# x_proba=[[0 for i in range(num_posbl_x)] for j in range(boot_num)]
# compute_x_proba()
# print(sum(x_proba[0]))

In [ ]:
# # importing the required module
# import matplotlib.pyplot as plt
  
 
# # plotting the points 
# plt.plot(w_avg, ll)
  
# # naming the x axis
# plt.xlabel('x - axis')
# # naming the y axis
# plt.ylabel('y - axis')
  
# # giving a title to my graph
# plt.title('My first graph!')
  
# # function to show the plot
# plt.show()

In [ ]:
# Import module
from tabulate import tabulate

# Assign data
mydata = [
    ["Full Sample", fs_mean, fs_var],
    ["Complete Case", cc_mean, cc_var],
    ["Fractional Imputation", fi_mean, fi_var],
]
 
# Create header
head = ["Estimator", "Mean", "Variance"]
 
# Display table
print(tabulate(mydata, headers=head, tablefmt="grid"))

+-----------------------+-------------------------+-------------------------+
| Estimator             | Mean                    | Variance                |
+=======================+=========================+=========================+
| Full Sample           | [0.5047 0.5047]         | [0.00049779 0.00049779] |
+-----------------------+-------------------------+-------------------------+
| Complete Case         | [0.50536383 0.50213682] | [0.0006267  0.00085047] |
+-----------------------+-------------------------+-------------------------+
| Fractional Imputation | [0.49280866 0.49169841] | [0.00056755 0.00058825] |
+-----------------------+-------------------------+-------------------------+
